In [2]:
import numpy as np
import random

def create_old_distance_matrix(pitch_sets):
    distance_matrix = []
    for i in range(len(pitch_sets)):
        row = []
        for j in range(len(pitch_sets)):
            row.append(len(list_intersection(pitch_sets[i],pitch_sets[j])))
        row_sum = sum(row)
        row = np.array(row,dtype='f')/row_sum
        row = np.cumsum(row)
        distance_matrix.append(list(row))
    return(distance_matrix)

def list_intersection(list1, list2):
    list3 = [value for value in list1 if value in list2]
    return list3

def remainder_after_intersection(list1, list2):
    list3 = [x for x in list1 if x not in list2]
    list4 = [x for x in list2 if x not in list1]
    return list3, list4

def closest_element_to_input(list1, myNumber):
    return min(list1, key=lambda x:abs(x-myNumber))

def create_new_distance_matrix(pitch_sets):
    distance_matrix = []
    for i in range(len(pitch_sets)):
        row = []
        for j in range(len(pitch_sets)):
            ##comparison algorithm changed
            static_set = pitch_set[i]
            comp_set = pitch_sets[j]
            static_leftover, comp_leftover = remainder_after_intersection(static_set, comp_set)
            ##if the remainder is empty for both (i.e. they are the same set), give a value of 1
            if not static_leftover and not comp_leftover:
                ## score = 0 means it could never go to itself -- might be worth considering
                ## score = 100 means very low probability
                ## score = 1 means very high probability
                score = 1
            else:
                ## if the two leftovers are of different sizes -- what should the score be?
                ## Case1: extra notes are treated as nothing (generating voices does not take energy)
                ## Case2: extra notes need to find the nearest neighbor to collapse or expand to (all voices are maintained)
                ## Case3: unbalanced; if static > comp, note off events (losing a voice) don't take energy;
                ##        if comp > static, note on events (adding a voice) do take energy and need to have a neighbor to come from
                ## Case3 can be flipped if desired
                ##
                ## I think these rules make sense: if you can maintain common tones, do so. If you have leftover notes, existing voices
                ## should be kept existing. This means that even if there is a note in set1 that is closer to a remainder note in set2
                ## but it is not in the remainder of set1, then the leftover voices of both sets should be used to calculate distance.
                ## The only cases extenuating for this rule is if there is an addition or subtraction of an additional voice. In this case,
                ## any note in the opposite set can be used to "split" or "collapse" the voice.
                ## 
                ## It is important to note that this system cannot be extended to different voices of chords at the moment. For
                ## now, it assumes all chords are created equal in terms of voicing and instead cares about the collection of notes
                ## more than how they are sounded. This is definitely something left to future work, as it would make more sense to
                ## accomodate different voicings of chords (the distance function could be calculate simply by measuring the distance)
                ## in frequency as opposed to divisions of the octave.
                ##
                ## Another important note is that the assumptions of this model assume equal distance within different divisions of 
                ## different spaces. This is obviously not the case but is used as a jumping off point for the building of this system.
                ##
                ## all that being said, here is the implementation of the most rudimentary version of this algorithm
                if len(static_leftover) == len(comp_leftover):
                    ## nuance of being equal in length: if two notes are equidistant from one note, or if they are both close
                    ## to one note and not the other, how do we ensure the lowest possible score
                    for item in static_leftover:
                        return 0
                
            row.append(len(pitch_sets[i].intersection(pitch_sets[j])))
        row_sum = sum(row)
        row = np.array(row,dtype='f')/row_sum
        row = np.cumsum(row)
        distance_matrix.append(list(row))
    return(distance_matrix)

#sublists are not in normal form (i.e., the same intervals can be allowed)
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

#created limited sets based on specific axes
def limitedsets(max_int, axes, vertices):
    if len(axes) != vertices:
        return "You inputed things wrong! Make sure axes and vertices are the same"
    else:
        limited_sets = []
        first_set = [1]
        for i in range(1, len(axes)):
            first_set.append(first_set[i-1] + axes[i-1])
            
        limited_sets.append(first_set)
        for i in range(1, max_int):
            new_set = [(element + i) % max_int for element in first_set]
            limited_sets.append(new_set)
    return limited_sets

def create_drunkards_walking_music(pitch_sets, distance_matrix):
    current_set = random.randint(0,len(pitch_sets)-1)
    drunkards_walking_music = [pitch_sets[current_set]]
    for i in range(ITERATIONS):
        r = random.uniform(0,1)
        j = 0
        while distance_matrix[current_set][j] < r:
            j +=1
        drunkards_walking_music.append(list(pitch_sets[j]))
        current_set = j
    return(drunkards_walking_music)

##Self defined flatten function for ease of use
def flatten(t):
    return [item for sublist in t for item in sublist]

# Implementation 1
## Each shape has all possible subsets of frequency division of size VERTICES

In [3]:
import itertools

##Pentagon
pVERTICES = 5
pDIVISIONS = 15

p_master_set = np.arange(1,pDIVISIONS+1)

p_subsets = findsubsets(p_master_set, pVERTICES)

p_limited_subsets = limitedsets(max(p_master_set), [1,2,3,4,5], pVERTICES)
#print(p_limited_subsets)

##Diamond
dVERTICES = 4
dDIVISIONS = [8, 10, 12, 14, 16]

d_master_set1 = np.arange(1,dDIVISIONS[0])
d_master_set2 = np.arange(1,dDIVISIONS[1])
d_master_set3 = np.arange(1,dDIVISIONS[2])
d_master_set4 = np.arange(1,dDIVISIONS[3])
d_master_set5 = np.arange(1,dDIVISIONS[4])

diamond_subsets1 = findsubsets(d_master_set1, dVERTICES)
diamond_subsets2 = findsubsets(d_master_set2, dVERTICES)
diamond_subsets3 = findsubsets(d_master_set3, dVERTICES)
diamond_subsets4 = findsubsets(d_master_set4, dVERTICES)
diamond_subsets5 = findsubsets(d_master_set5, dVERTICES)

d_limited_subsets1 = limitedsets(max(d_master_set1), [1,3,1,3], dVERTICES)
d_limited_subsets2 = limitedsets(max(d_master_set2), [1,4,1,4], dVERTICES)
d_limited_subsets3 = limitedsets(max(d_master_set3), [2,4,2,4], dVERTICES)
d_limited_subsets4 = limitedsets(max(d_master_set4), [5,2,5,2], dVERTICES)
d_limited_subsets5 = limitedsets(max(d_master_set5), [3,5,3,5], dVERTICES)
#print(diamond_limited_subsets5)

##Boat
bVERTICES = 7
bDIVISIONS = [17, 19, 21, 23, 25]

b_master_set1 = np.arange(1,bDIVISIONS[0])
b_master_set2 = np.arange(1,bDIVISIONS[1])
b_master_set3 = np.arange(1,bDIVISIONS[2])
b_master_set4 = np.arange(1,bDIVISIONS[3])
b_master_set5 = np.arange(1,bDIVISIONS[4])

b_subsets1 = findsubsets(b_master_set1, bVERTICES)
b_subsets2 = findsubsets(b_master_set2, bVERTICES)
b_subsets3 = findsubsets(b_master_set3, bVERTICES)
b_subsets4 = findsubsets(b_master_set4, bVERTICES)
b_subsets5 = findsubsets(b_master_set5, bVERTICES)

b_limited_subsets1 = limitedsets(max(b_master_set1), [2,1,3,1,4,1,5], bVERTICES)
b_limited_subsets2 = limitedsets(max(b_master_set2), [3,2,4,2,5,2,1], bVERTICES)
b_limited_subsets3 = limitedsets(max(b_master_set3), [4,3,5,3,1,3,2], bVERTICES)
b_limited_subsets4 = limitedsets(max(b_master_set4), [5,4,1,4,2,4,3], bVERTICES)
b_limited_subsets5 = limitedsets(max(b_master_set5), [1,5,2,5,3,5,4], bVERTICES)
#print(boat_limited_subsets1)

##Star
sVERTICES = 10
sDIVISIONS = 30

s_master_set = np.arange(1,sDIVISIONS+1)

s_subsets = findsubsets(s_master_set, sVERTICES)

s_limited_subsets = limitedsets(max(s_master_set), [4,3,5,4,1,5,2,1,3,2], sVERTICES)
#print(s_limited_subsets)

In [4]:
## create distance matrices for just limited subsets, using average divisions for both diamonds and boats
##THIS USES OLD ALGORITHM FOR DISTANCE

from scipy.io import savemat

##distance matrices
p_distance_matrix = create_old_distance_matrix(p_limited_subsets)
d_distance_matrix3 = create_old_distance_matrix(d_limited_subsets3)
b_distance_matrix3 = create_old_distance_matrix(b_limited_subsets3)
s_distance_matrix = create_old_distance_matrix(s_limited_subsets)

ITERATIONS = 5

##chord progression
p_progression = create_drunkards_walking_music(p_limited_subsets, p_distance_matrix)
d_progression = create_drunkards_walking_music(d_limited_subsets3, d_distance_matrix3)
b_progression = create_drunkards_walking_music(b_limited_subsets3, b_distance_matrix3)
s_progression = create_drunkards_walking_music(s_limited_subsets, s_distance_matrix)

mdic = {"p_progression": p_progression, "d_progression": d_progression, 
        "b_progression": b_progression, "s_progression": s_progression}

savemat("penrose_progressions.mat", mdic)

In [5]:
from IPython.display import Audio
import soundfile as sf

##GLOBAL PARAMETERS
NUM_CHANNELS = 7
NUM_LFE = 1
GROUND_VALUES = [16.35, 32.70, 65.41, 130.81, 261.63]
TIME_FIRMUS = [5, 4, 3, 2, 1, 1, 1, 2, 3, 4, 5] #proportion of total duration
DURATION = 5 #mins
FS = 48000


##build audio files
##pentagon audio files
sorted_p = [sorted(item) for item in p_progression]
for ground in GROUND_VALUES:
    for i in range(len(sorted_p)):
        pitch_set = sorted_p[i]
        durations = {}
        for j in range(len(pitch_set)):
            durations[j] = random.sample(range(5, 10), 1)[0]
        max_dur = max(durations.values())
        for w in range(len(pitch_set)):
            pitch = pitch_set[w]
            duration = durations[w]
            time = np.arange(0, duration, 1/FS)
            window = np.hamming(len(time))
            amp = 1/pitch
            wave = amp*window*np.sin(2*np.pi*pitch*ground*time)
            padding = max_dur*FS - duration*FS
            if padding != 0:
                front_pad = random.sample(range(0, int(np.floor(padding/2))),1)[0]
                wave = np.pad(wave, (front_pad, padding-front_pad))
            
            filepath = str(ground)+ "pent" + str(i) + "note" + str(w+1) + ".wav"
            sf.write(filepath, wave, FS)
        

<ipython-input-5-f76bc46774dd>:28: RuntimeWarning: divide by zero encountered in long_scalars
  amp = 1/pitch
<ipython-input-5-f76bc46774dd>:29: RuntimeWarning: invalid value encountered in multiply
  wave = amp*window*np.sin(2*np.pi*pitch*ground*time)


Instead of exporting every single shape collection based on the markov model, instead, the markov model will be used in Matlab. All I need for Reaper (i.e. audio files) are each frequency at some duration for each division of the frequency space used. That's what I'll be doing below.

So I realized that since I'm doing things spectrally, all I need to compute are the audio files for the star. This will cover all the other shapes' notes, since it's not a limiting of the frequency space but rather an additive exploration of it. It's a bit old school, frankly, and I probably could benefit from implementing my initial idea. But since time is pressing, we will do this for now.

In [6]:
GROUND_VALUES = [16.35, 32.70, 65.41, 130.81, 261.63]
FS = 48000

from IPython.display import Audio
import soundfile as sf

##pentagon notes
for ground in GROUND_VALUES:
    for i in range(len(s_master_set)):
        pitch = s_master_set[i]
        frequency = ground*pitch
        duration = 10
        #while (duration < 10*FS):
            #duration += frequency
        #duration = np.round(duration/10)
        time = np.arange(0, duration, 1/FS)
        window = np.hamming(len(time))
        amp = 1/pitch
        wave = amp*window*np.sin(2*np.pi*pitch*ground*time)
            
        filepath = str(ground)+ "note" + str(i+1) + ".wav"
        sf.write(filepath, wave, FS)

In [7]:
FS = 48000

duration = 0
frequency = 110
while (duration < 10*FS):
    duration += frequency
duration = np.round(duration/10)
time = np.arange(0, duration, 1/FS)
time2 = np.arange(0, 10, 1/FS)
print(time[0:100])
print(time2[0:100])

[0.00000000e+00 2.08333333e-05 4.16666667e-05 6.25000000e-05
 8.33333333e-05 1.04166667e-04 1.25000000e-04 1.45833333e-04
 1.66666667e-04 1.87500000e-04 2.08333333e-04 2.29166667e-04
 2.50000000e-04 2.70833333e-04 2.91666667e-04 3.12500000e-04
 3.33333333e-04 3.54166667e-04 3.75000000e-04 3.95833333e-04
 4.16666667e-04 4.37500000e-04 4.58333333e-04 4.79166667e-04
 5.00000000e-04 5.20833333e-04 5.41666667e-04 5.62500000e-04
 5.83333333e-04 6.04166667e-04 6.25000000e-04 6.45833333e-04
 6.66666667e-04 6.87500000e-04 7.08333333e-04 7.29166667e-04
 7.50000000e-04 7.70833333e-04 7.91666667e-04 8.12500000e-04
 8.33333333e-04 8.54166667e-04 8.75000000e-04 8.95833333e-04
 9.16666667e-04 9.37500000e-04 9.58333333e-04 9.79166667e-04
 1.00000000e-03 1.02083333e-03 1.04166667e-03 1.06250000e-03
 1.08333333e-03 1.10416667e-03 1.12500000e-03 1.14583333e-03
 1.16666667e-03 1.18750000e-03 1.20833333e-03 1.22916667e-03
 1.25000000e-03 1.27083333e-03 1.29166667e-03 1.31250000e-03
 1.33333333e-03 1.354166

In [8]:
def create_new_distance_matrix(pitch_sets):
    distance_matrix = []
    for i in range(len(pitch_sets)):
        row = []
        static_set = pitch_set[i]
        for j in range(len(pitch_sets)):
            ##comparison algorithm changed
            comp_set = pitch_sets[j]
            static_leftover, comp_leftover = remainder_after_intersection(static_set, comp_set)
            ##if the remainder is empty for both (i.e. they are the same set), give a value of 1
            if not static_leftover and not comp_leftover:
                ## score = 0 means it could never go to itself -- might be worth considering
                ## score = 100 means very low probability
                ## score = 1 means very high probability
                score = 1
            else:
                ## if the two leftovers are of different sizes -- what should the score be?
                ## Case1: extra notes are treated as nothing (generating voices does not take energy)
                ## Case2: extra notes need to find the nearest neighbor to collapse or expand to (all voices are maintained)
                ## Case3: unbalanced; if static > comp, note off events (losing a voice) don't take energy;
                ##        if comp > static, note on events (adding a voice) do take energy and need to have a neighbor to come from
                ## Case3 can be flipped if desired
                ##
                ## I think these rules make sense: if you can maintain common tones, do so. If you have leftover notes, existing voices
                ## should be kept existing. This means that even if there is a note in set1 that is closer to a remainder note in set2
                ## but it is not in the remainder of set1, then the leftover voices of both sets should be used to calculate distance.
                ## The only cases extenuating for this rule is if there is an addition or subtraction of an additional voice. In this case,
                ## any note in the opposite set can be used to "split" or "collapse" the voice.
                ## 
                ## It is important to note that this system cannot be extended to different voices of chords at the moment. For
                ## now, it assumes all chords are created equal in terms of voicing and instead cares about the collection of notes
                ## more than how they are sounded. This is definitely something left to future work, as it would make more sense to
                ## accomodate different voicings of chords (the distance function could be calculate simply by measuring the distance)
                ## in frequency as opposed to divisions of the octave.
                ##
                ## Another important note is that the assumptions of this model assume equal distance within different divisions of 
                ## different spaces. This is obviously not the case but is used as a jumping off point for the building of this system.
                ##
                ## all that being said, here is the implementation of the most rudimentary version of this algorithm
                if len(static_leftover) == len(comp_leftover):
                    ## nuance of being equal in length: if two notes are equidistant from one note, or if they are both close
                    ## to one note and not the other, how do we ensure the lowest possible score
                    for item in static_leftover:
                        return 0
                
            row.append(len(pitch_sets[i].intersection(pitch_sets[j])))
        row_sum = sum(row)
        row = np.array(row,dtype='f')/row_sum
        row = np.cumsum(row)
        distance_matrix.append(list(row))
    return(distance_matrix)

In [9]:
import numpy as np

def find_map_and_score(set1, set2, TYPE, division = 12):
    #make sure smallest set is used for comparisons
    reverse_indicator = 1
    if len(set1) > len(set2):
        holder = set1[:]
        set1 = set2[:]
        set2 = holder[:]
        reverse_indicator = -1
        
    ##Initiliaze outputs
    mappings = []
    score = 0
    
    #don't overwrite set2
    comparison_set = set2[:]
    
    for note in set1:
        smallest_distance = np.inf
        for dest in comparison_set:
            #distance function needs to be defined for specific input and units
            dist = distance(note, dest, TYPE, division = division)
            if dist < smallest_distance:
                smallest_distance = dist
                to_note = dest
        tup_out = (note, to_note)
        score += smallest_distance
        comparison_set.remove(tup_out[1])
        mappings.append(tup_out[::reverse_indicator])
        
    leftover = comparison_set
    return (mappings, score, leftover)    

In [10]:
set1 = [0, 1, 2]
set2 = [3, 4, 5]

def distance(note1, note2, note_type, division = 12):
    if note_type == "int":
        return min(abs(note1-note2), division - abs(note1-note2))
    elif note_type == "ratio":
        return abs(note1-note2)
    elif note_type == 'hertz':
        return abs(np.log2(note1) - np.log2(note2))

In [11]:
print(distance(0, 1, "int", division = 12))
print(distance(4/3, 1/1, "ratio"))
print(distance(200, 400, "hertz"))

1
0.33333333333333326
1.0000000000000009


In [12]:
find_map_and_score(set1, set2, "int", division = 12)

([(0, 3), (1, 4), (2, 5)], 9, [])

In [13]:
## this function assumes equal friction for adding and removing voices
import itertools
import numpy as np

def minimum_distance_mapping(set1, set2, TYPE, division = 12):
    ## set 1 should be smaller than set2 if different sizes
    reverse_indicator = 1
    if len(set1) > len(set2):
        holder = set1[:]
        set1 = set2[:]
        set2 = holder[:]
        reverse_indicator = -1
    maps_and_scores = []
    set1_orderings = list(itertools.permutations(set1))
    for combination in set1_orderings:
        combination = list(combination)
        maps_and_scores.append(find_map_and_score(combination, set2, TYPE, division=division))
    
    lowest_score = np.inf
    for map_and_score in maps_and_scores:
        current_score = map_and_score[1]
        current_map = map_and_score[0]
        current_leftover = map_and_score[2]
        if current_score < lowest_score:
            lowest_score = current_score
            best_map = current_map
            leftover = current_leftover
    
    ## this is where it gets a little bit tricky again
    ## should I have the leftover notes be allowed to go to the same note in the other set?
    ## or should we only allow one additional voice be connected to a shared note
    ## the latter seems more reasonable, but theoretically both are possible
    
    ## I will do the latter for now but the difference in code is just changing one call to 
    ## find_map_and_score to multiple individual calls
    ## more efficient to do the latter actually, but could be a cooler compositional effect doing the former
    ## actually it's more efficient to do the former, as the latter requires more permutations to be done
    ## but that is okay for now
    
    if len(leftover) != 0:
        while len(best_map) != len(set2):
            leftover_orderings = list(itertools.permutations(leftover))
            leftover_maps_and_scores = []
            for combination in leftover_orderings:
                combination = list(combination)
                leftover_maps_and_scores.append(find_map_and_score(combination, set1, TYPE, division = division))

            leftover_lowest = np.inf
            for leftover_map_and_score in leftover_maps_and_scores:
                leftover_map = leftover_map_and_score[0]
                leftover_score = leftover_map_and_score[1]
                leftover_leftover = leftover_map_and_score[2]
                if leftover_score < leftover_lowest:
                    leftover_lowest_score = leftover_score
                    best_leftover_map = [tup[::-1] for tup in leftover_map]
                    leftover = leftover_leftover
            best_map = best_map + best_leftover_map
            lowest_score += leftover_lowest_score
    
    final_map = [tup[::reverse_indicator] for tup in best_map]
    final_score = lowest_score 
    
    return (final_map, final_score)

In [14]:
set1 = [0,]
set2 = [1, 4, 5, 7, 9]
TYPE = "int"

minimum_distance_mapping(set1, set2, TYPE)

[(4, 5, 7, 9), (4, 5, 9, 7), (4, 7, 5, 9), (4, 7, 9, 5), (4, 9, 5, 7), (4, 9, 7, 5), (5, 4, 7, 9), (5, 4, 9, 7), (5, 7, 4, 9), (5, 7, 9, 4), (5, 9, 4, 7), (5, 9, 7, 4), (7, 4, 5, 9), (7, 4, 9, 5), (7, 5, 4, 9), (7, 5, 9, 4), (7, 9, 4, 5), (7, 9, 5, 4), (9, 4, 5, 7), (9, 4, 7, 5), (9, 5, 4, 7), (9, 5, 7, 4), (9, 7, 4, 5), (9, 7, 5, 4)]
[(7, 5, 4), (7, 4, 5), (5, 7, 4), (5, 4, 7), (4, 7, 5), (4, 5, 7)]
[(5, 7), (7, 5)]
[(5,)]


([(0, 1), (0, 9), (0, 4), (0, 7), (0, 5)], 18)